# ECG Review

In [1]:
from logparser import LogReader
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, RangeTool, BoxZoomTool
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
import pandas as pd
import numpy as np

output_notebook()

Loading BokehJS ...

In [2]:
filepath = "./.data/log_mimir_20230715122058.txt"

log = LogReader(filepath, ['PPG'])

#print(log.health)

In [3]:
# Turn to time serie
ecg_ts = log.health.set_index('datetime')

meas = "value_0"

# make derivative
#ecg_ts['d_value_1'] = ecg_ts['value_0'] - ecg_ts['value_0'].shift(-1)

# # Remove outliers
# q = ecg_ts[meas].quantile(0.99)
# ecg_ts = ecg_ts[ecg_ts[meas] < q] 

# plot
source = ColumnDataSource(ecg_ts)
# p = figure(x_axis_type="datetime", width=800, height=350)
# p.line('datetime', 'd_value_1', source=source)
# show(p)

dt = ecg_ts.index[100]

p = figure(height=300, width=1200,
           x_axis_type="datetime", x_axis_location="above",
           background_fill_color="#efefef", x_range=(ecg_ts.index[100], ecg_ts.index[1000]))

p.line('datetime', meas, source=source)
p.yaxis.axis_label = meas

select = figure(title="Drag the middle and edges of the selection box to change the range above",
                height=130, width=1200, y_range=p.y_range,
                x_axis_type="datetime", y_axis_type=None, toolbar_location=None, background_fill_color="#efefef")

range_tool = RangeTool(x_range=p.x_range)
range_tool.overlay.fill_color = "navy"
range_tool.overlay.fill_alpha = 0.2

select.line('datetime', meas, source=source)
select.ygrid.grid_line_color = None
select.add_tools(range_tool)
select.add_tools(BoxZoomTool(dimensions="width"))
select.toolbar.active_multi = 'auto'

show(column(p, select))
